# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

*Изучим данные*

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
#Проверим дубликаты
df.duplicated().sum()

0

**Вывод:**
предобработки данных не требуется, так как она была проведена ранее; пропусков и дубликатов нет.


## Разбейте данные на выборки

*Извлечем признаки и целевой признак*

In [ ]:
features = df.drop(['is_ultra'], axis=1) # признаки
target = df['is_ultra'] # целевой признак

*Разделим данные на обучающую, валидационную и тестовую выборки 3:1:1*

In [ ]:
# отделяем тестовую выборку
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)
# выделяем из валидационной обучающую
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.2, random_state=12345)

*Выведем размеры выборок*

In [ ]:
# размер обучающей выборки
print(features_train.shape)
print(target_train.shape)

(2056, 4)
(2056,)


In [ ]:
# размер валидационной выборки
print(features_valid.shape)
print(target_valid.shape)

(515, 4)
(515,)


In [ ]:
# размер тестовой выборки
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


**На этом этапе извлекли признаки и целевой признак из исходных данных, и разделили данные на обучающую, валидационную и тестовую выборки 3:1:1 (60%, 20% и 20% соответственно).**

## Исследуйте модели

### Решающее дерево

*Найдём наилучшую модель. Переберем гиперпараметр 'max_depth' и оценим качество модели на валидационной выборке.*

In [ ]:
best_model_dc = None
best_depth_dc = 0
best_result_dc = 0
for depth in range(1, 11):
    model_dc = DecisionTreeClassifier(random_state=12345, max_depth = depth).fit(features_train, target_train)
    predictions_valid_dc = model_dc.predict(features_valid)
    result_dc = accuracy_score(target_valid, predictions_valid_dc)
    if best_result_dc < result_dc:
        best_model_dc = model_dc
        best_depth_dc = depth
        best_result_dc = result_dc

print('Глубина дерева:', best_depth_dc)
print('Качество модели:', best_result_dc)


Глубина дерева: 7
Качество модели: 0.7650485436893204


**Вывод:** Accuracy лучшей модели дерева решений из десяти равно 0.7650485436893204, с гиперпараметром max_depth = 7.

### Случайный лес

*Найдём наилучшую модель. Переберем гиперпараметры 'n_estimators' и 'max_depth', и оценим качество модели на валидационной выборке.*

In [ ]:
best_model_rc = None
best_est_rc = 0
best_depth_rc = 0
best_result_rc = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_rc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth).\
        fit(features_train, target_train)
        predictions_valid_rc = model_rc.predict(features_valid)
        result_rc = accuracy_score(target_valid, predictions_valid_rc)
            if best_result_rc < result_rc:
            best_model_rc = model_rc
            best_est_rc = est
            best_depth_rc = depth
            best_result_rc = result_rc
print('Количество деревьев:', best_est_rc)
print('Глубина дерева:', best_depth_rc)
print('Качество модели:', best_result_rc)

**Вывод:** Accuracy лучшей модели из десяти равно 0.7922330097087379, с гиперпараметром n_estimators = 6 и max_depth = 9,

### Логистическая регрессия

*Найдём наилучшую модель. Переберем гиперпараметры 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga', и оценим качество модели на валидационной выборке.*

In [ ]:
best_model_lr = None
best_result_lr = 0
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for s in solver:
    model_lr = LogisticRegression(random_state=12345, solver=s, max_iter=10000).fit(features_train, target_train)
    predictions_valid_lr = model_lr.predict(features_valid)
    result_lr = accuracy_score(target_valid, predictions_valid_lr)
    if best_result_lr < result_lr:
        best_model_lr = model_lr
        best_result_lr = result_lr
        best_s = s

print('Качество модели:', best_result_lr)
print('Лучший гиперпараметр:', best_s)

/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Качество модели: 0.7184466019417476
Лучший гиперпараметр: liblinear


**Вывод:** Accuracy лучшей модели равно 0.7184466019417476 с гиперпараметром liblinear.

**Вывод:**
* наилучший результат качества у модели Случайный лес - RandomForestClassifier -  0.7922330097087379;
* наилучший результат качества у модели Решающее дерево - DecisionTreeClassifier - 0.7650485436893204;
* наилучший результат качества у модели Логистическая регрессия - LogisticRegression -  0.7184466019417476.


## Проверьте модель на тестовой выборке

*Наиболее эффективно себя показала модель случайного леса - RandomForestClassifier, поэтому проверим ее на тестовой выборке.*

In [ ]:
model_rc = RandomForestClassifier(random_state=12345, n_estimators = 6, max_depth = 9).fit(features_train, target_train)
predictions_test_rc = model_rc.predict(features_test)
result_test_rc = accuracy_score(target_test, predictions_test_rc)
print('Качество модели:', result_test_rc)

Качество модели: 0.7947122861586314


**Вывод:** Accuracy модели на тестовой выборке равно 0.7947122861586314.

## (бонус) Проверьте модели на адекватность

*Для проверки используем модель DummyClassifier.*

In [ ]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Качество DummyClassifier:', result_dc)
print('Качество RandomForestClassifier:', result_test_rc)

Качество DummyClassifier: 0.6757281553398058
Качество RandomForestClassifier: 0.7947122861586314


**Вывод:** качество случайной модели DummyClassifier ниже, чем у модели RandomForestClassifier - модель адекватна и эффективна.

## **Общий вывод:**
* исследованы модели Decision Tree Classifier, RandomForestClassifier и Logistic Regression;
* наилучшую эффективность показала модель RandomForestClassifier с гиперпараметрами n_estimators равным = 6 и max_depth = 9;
* модель RandomForestClassifier с наилучшими гиперапараметрами проверена на тестовой выборке и показала accuracy = 0.7947122861586314;
* также модель RandomForestClassifier проверена на вменяемость, для проверки использовалась модель DummyClassifier, которая показала результат accuracy = 0.7060653188180405, что ниже, чем у тестовой модели.